In [1]:
# add dependencies
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import time

from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.ext.automap import automap_base

from sqlalchemy import Column, Integer, String, Float, and_, Date, desc, func

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

from flask import Flask, jsonify

#################################################
# Database Setup
#################################################
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Create our session (link) from Python to the DB
session = Session(engine)

# Save references to the invoices and invoice_items tables
Station = Base.classes.stations
Measurement = Base.classes.measurements

#################################################
# Flask Setup
#################################################
app = Flask(__name__)


@app.route("/")
def welcome():

     return (
         f"Avalable Routes:<br/>"
         f"/api/v1.0/precipitation"
         f"- Dates and temperature observations from the last year<br/>"

         f"/api/v1.0/stations"
         f"- List of stations<br/>"

         f"/api/v1.0/tobs"
         f"- Temperature Observations from the past year<br/>"

         f"/api/v1.0/<start>"
         f"- Minimum temperature, the average temperature, and the max temperature for a given start day<br/>"

         f"/api/v1.0/<start>/<end>"
         f"- Minimum temperature, the average temperature, and the max temperature for a given start-end range<br/>"
     )

@app.route("/api/v1.0/precipitation")
def pcrp():
    # create the date range, use today as the api here does not take a start/end date
    today = datetime.datetime.today()
    today = today.date()
    last_year = today - datetime.timedelta(365)
    pcp_year = session.query(Measurement.date, Measurement.prcp).filter(and_(Measurement.date <= today, Measurement.date >= last_year)).all()
    return jsonify(pcp_year)

@app.route("/api/v1.0/stations")
def station_list():
    st_list = session.query(Station.station).all()

    all_stations= list(np.ravel(st_list))

    return jsonify(all_stations)

@app.route("/api/v1.0/tobs")
def temp_year():
    # create the date range, use today as the api here does not take a start/end date
    today = datetime.datetime.today()
    today = today.date()
    last_year = today - datetime.timedelta(365)
    temp_year = session.query(Measurement.date, Measurement.tobs).filter(and_(Measurement.date <= today, Measurement.date >= last_year)).all()

    return jasonify(temp_year)

@app.route("/api/v1.0/<start>")
def start_temp(start):
    # get the min/avg/max
    temp_data = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).filter(Measurement.date >= start).all()
    
    return jsonify(temp_data)

    

@app.route("/api/v1.0/<start>/<end>")
def range_temp(start, end):
 # get the min/avg/max
    temp_data = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).filter(and_(Measurement.date >= start, Measurement.date <= end)).all()
    
    return jsonify(temp_data)

if __name__ == '__main__':
    app.run(debug=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Apr/2018 16:38:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2018 16:38:58] "GET /api/v1.0/precipitation HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2018 16:39:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2018 16:39:30] "GET /api/v1.0/stations HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2018 16:39:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2018 16:39:58] "GET /api/v1.0/ HTTP/1.1" 404 -
127.0.0.1 - - [24/Apr/2018 16:40:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2018 16:40:48] "GET /api/v1.0/precipitation HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2018 16:41:23] "GET /api/v1.0/2017-06-29 HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2018 16:41:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2018 16:42:24] "GET /api/v1.0/2017-6-29/2017-08-03 HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2018 16:42:32] "GET /api/v1.0/2017-06-29/2017-08-03 HTTP/1.1" 200 -
